In [342]:
import hmac
import hashlib
import base64
msg = 'ana weza'
msg = msg.encode()
dig = hmac.new(b'1234567890', msg=msg, digestmod=hashlib.sha256).digest()
base64.b64encode(dig).decode()      # py3k-mode
import json
from web3 import Web3, HTTPProvider
from web3.contract import ConciseContract
from cryptography.fernet import Fernet
import pickle
from datetime import datetime
from eth_account.messages import encode_defunct
import threading

## publishing the contract

In [426]:
# web3.py instance
w3 = Web3(HTTPProvider("HTTP://127.0.0.1:7545"))
print(w3.isConnected())

key="332a4857af5d050825cc0b6572e00695078731c2877288be4e3c9daf9c8c2d87"
acct = w3.eth.account.privateKeyToAccount(key)

# compile your smart contract with truffle first
truffleFile = json.load(open(r'C:\Users\mo7amad mo3taz\Desktop\security project\python_edition\pythonDapp\build\contracts\Greeter.json'))
abi = truffleFile['abi']
bytecode = truffleFile['bytecode']
contract= w3.eth.contract(bytecode=bytecode, abi=abi)

#building transaction
construct_txn = contract.constructor().buildTransaction({
    'from': acct.address,
    'nonce': w3.eth.getTransactionCount(acct.address),
    'gas': 3000000,
    'gasPrice': w3.toWei('21', 'gwei')})

signed = acct.signTransaction(construct_txn)

tx_hash=w3.eth.sendRawTransaction(signed.rawTransaction)
print(tx_hash.hex())
tx_receipt = w3.eth.waitForTransactionReceipt(tx_hash)
contract_address = tx_receipt['contractAddress']
print("Contract Deployed At:", tx_receipt['contractAddress'])


True
0x7b48ca61e90bcf7fbd33d2976f87e3a4dfe1ec98517d67347cd84fec02caaa29
Contract Deployed At: 0x19ad2458b793256b3cBB39d0D0Bb44873652F1E1


### getting a contract instance

In [427]:
truffleFile = json.load(open(r'C:\Users\mo7amad mo3taz\Desktop\security project\python_edition\pythonDapp\build\contracts\Greeter.json'))
abi = truffleFile['abi']
bytecode = truffleFile['bytecode']

w3 = Web3(HTTPProvider("HTTP://127.0.0.1:7545")) #modify
print(w3.isConnected())
contract_address = Web3.toChecksumAddress(contract_address) #modify
contract = w3.eth.contract(abi=abi, bytecode=bytecode)
contract_instance = w3.eth.contract(abi=abi, address=contract_address)
print(contract_instance.functions.greet().call())

True
Hello


### defining the clinic entity

In [389]:
def load_enc_key(enc_path):
        with open(enc_path, "rb") as infile:
 	         return pickle.load(infile)

def load_count(count_path):
        with open(count_path, "rb") as infile:
 	         return pickle.load(infile)

def write_count(path, count):
        with open(path, "wb") as outfile:
 	# "wb" argument opens the file in binary mode
	        pickle.dump(count, outfile)

In [390]:
def store_count(path,count):
    with open(path, "wb") as outfile:
	    pickle.dump(count, outfile)
    
def to_32byte_hex(val):
   return Web3.toHex(Web3.toBytes(val).rjust(32, b'\0'))

In [432]:
class clinic:
    def __init__ (self,clinic_num):
        
        if clinic_num == 0:
            self.address = '0x3ba20A3D2b870BC377F47AE68dAE0845ed2c2fF4'
            self.private_key = '150114451a67123aeb55ca8f37bbe719b6eabb7bb29923596b643c74abe83d16'
            self.enc_key = load_enc_key(r'C:\Users\mo7amad mo3taz\Desktop\security project\python_edition\c1_enc.pickle')
            self.count_path = r'C:\Users\mo7amad mo3taz\Desktop\security project\python_edition\c1_count.pickle'
            self.patient_count = load_count(self.count_path)
            self.v_vount_path = r'C:\Users\mo7amad mo3taz\Desktop\security project\python_edition\c1_v_count.pickle'
            self.v_count = load_count(self.count_path)

        elif clinic_num == 1:
            self.address = '0x255d58708D85b74fF56Fd2ba31BA23E04bd83163'
            self.private_key = 'c6bed74a4d13163f280f8bd60c6d6f40428ee86eccb681689c74d30883b4abde'
            self.enc_key = load_enc_key(r'C:\Users\mo7amad mo3taz\Desktop\security project\python_edition\c2_enc.pickle')
            self.count_path = r'C:\Users\mo7amad mo3taz\Desktop\security project\python_edition\c2_count.pickle'
            self.patient_count = load_count(self.count_path)
            self.v_vount_path = r'C:\Users\mo7amad mo3taz\Desktop\security project\python_edition\c2_v_count.pickle'
            self.v_count = load_count(self.count_path)

        else:
            self.address = '0x1216Fb7CA180FBCAb13593c69acFcA2F1AAd0fAD'
            self.private_key = '332a4857af5d050825cc0b6572e00695078731c2877288be4e3c9daf9c8c2d87'
            self.enc_key = load_enc_key(r'C:\Users\mo7amad mo3taz\Desktop\security project\python_edition\c3_enc.pickle')
            self.count_path = r'C:\Users\mo7amad mo3taz\Desktop\security project\python_edition\c3_count.pickle'
            self.patient_count = load_count(self.count_path)
            self.v_vount_path = r'C:\Users\mo7amad mo3taz\Desktop\security project\python_edition\c3_v_count.pickle'
            self.v_count = load_count(self.count_path)

        self.encrypter_decrypter = Fernet(self.enc_key)
        


    def sign_packet(self, packet):
        msgHash = Web3.keccak(text = packet)
        msgHash = encode_defunct(msgHash)
        signed_message = w3.eth.account.sign_message(msgHash, private_key=self.private_key)
        # print('signed_message: ',type(signed_message))
        return (signed_message.v, to_32byte_hex(signed_message.r), to_32byte_hex(signed_message.s))

    def add_new_patient(self, name, age, weight, height, sex, blood_p, pulse):
        now = datetime.now()
        dt_string = now.strftime("%d/%m/%Y %H:%M:%S") # time_stamp
        data = name +" # "+ age +" # "+ weight +" # "+ height +" # "+ sex +" # "+ blood_p +" # "+ pulse + " # " + dt_string
        data_enc = self.encrypter_decrypter.encrypt(data.encode())
        # print('original data: ',data)
        # print()
        # print('enc data: ',data_enc)
        # print()
        packet = str(self.patient_count)+ "#" + data_enc.decode('utf-8')
        self.patient_count +=1
        write_count(self.count_path,self.patient_count)
        # print('packet: ',packet)
        # print()
        signed_message = self.sign_packet(packet)
        hashed_message = Web3.keccak(text = packet)
        # print('hashed message: ',hashed_message)
        # print()
        tx_hash = contract_instance.functions.add_patient(packet, hashed_message, signed_message[0], signed_message[1], signed_message[2]).transact({
            'from': self.address,
            'nonce': w3.eth.getTransactionCount(self.address),
            'gas': 1728712,
            'gasPrice': w3.toWei('21', 'gwei')})
        w3.eth.waitForTransactionReceipt(tx_hash)
        # print('response: {}'.format(
        #     contract_instance.functions.greet().call()
        # ))
        print("success, patient id: ", self.patient_count-1)
        print('////////////////////////////////////')

    def regular_visit(self, patient_id, blood_p, pulse, reason, presc, follow_up):
            now = datetime.now()
            dt_string = now.strftime("%d/%m/%Y %H:%M:%S") # time_stamp
            data = blood_p +" # "+ pulse +" # "+ reason +" # "+ presc +" # "+ follow_up + " # " + dt_string + " # " + str(self.v_count)
            data_enc = self.encrypter_decrypter.encrypt(data.encode())
            print('original data: ',data)
            print()
            print('enc data: ',data_enc)
            print()
            packet = str(patient_id)+ "#" + data_enc.decode('utf-8')
            print('packet: ',packet)
            print()
            signed_message = self.sign_packet(packet)
            hashed_message = Web3.keccak(text = packet)
            print('hashed message: ',hashed_message)
            print()
            tx_hash = contract_instance.functions.add_reg_visit(packet, hashed_message, signed_message[0], signed_message[1], signed_message[2]).transact({
                'from': self.address,
                'nonce': w3.eth.getTransactionCount(self.address),
                'gas': 1728712,
                'gasPrice': w3.toWei('21', 'gwei')})
            w3.eth.waitForTransactionReceipt(tx_hash)
            # print('response: {}'.format(
            # contract_instance.functions.greet().call()
            # ))
            print('////////////////////////////////////')
            self.v_count+=1
            write_count(self.v_vount_path,self.v_count)

    def lab_visit(self, patient_id, blood_p, pulse, lab_test_typr, lab_test_result):
        now = datetime.now()
        dt_string = now.strftime("%d/%m/%Y %H:%M:%S") # time_stamp
        data = blood_p +" # "+ pulse +" # "+ lab_test_typr +" # "+ lab_test_result + " # " + dt_string + " # " + str(self.v_count)
        data_enc = self.encrypter_decrypter.encrypt(data.encode())
        print('original data: ',data)
        print()
        print('enc data: ',data_enc)
        print()
        packet = str(patient_id)+ "#" +data_enc.decode('utf-8')
        print('packet: ',packet)
        print()
        signed_message = self.sign_packet(packet)
        hashed_message = Web3.keccak(text = packet)
        print('hashed message: ',hashed_message)
        print()
        tx_hash = contract_instance.functions.add_lab_visit(packet, hashed_message, signed_message[0], signed_message[1], signed_message[2]).transact({
            'from': self.address,
            'nonce': w3.eth.getTransactionCount(self.address),
            'gas': 1728712,
            'gasPrice': w3.toWei('21', 'gwei')})
        w3.eth.waitForTransactionReceipt(tx_hash)
        # print('response: {}'.format(
        #     contract_instance.functions.greet().call()
        # ))
        print('////////////////////////////////////')
        self.v_count+=1
        write_count(self.v_vount_path,self.v_count)

    def view_records(self, patient_id):
        
        packet = patient_id
        # print('packet: ',packet)
        # print()
        signed_message = self.sign_packet(packet)
        # print('signed message: ', signed_message)
        # print()
        hashed_message = Web3.keccak(text = packet)
        # print('hashed message: ',hashed_message)
        # print()
        tx_hash = contract_instance.functions.define_records(packet, hashed_message, signed_message[0], signed_message[1], signed_message[2]).transact({
            'from': self.address,
            'nonce': w3.eth.getTransactionCount(self.address),
            'gas': 1728712,
            'gasPrice': w3.toWei('21', 'gwei')})
        w3.eth.waitForTransactionReceipt(tx_hash)
        all_records = contract_instance.functions.retrive_records().call()
        result = []
        for record in all_records:
            hed_dat = record.split("#")
            # print(hed_dat)
            if hed_dat[0] == patient_id:
                decrypted = self.encrypter_decrypter.decrypt(hed_dat[1].encode('utf-8'))
                result.append(decrypted.decode())
        print('records')
        print(result)

    def eval_view_all_records(self):
        all_records = contract_instance.functions.retrive_records1().call()
        print('clinic1 records:')
        for record in all_records:
            print(record)
            print()
        print()
        all_records = contract_instance.functions.retrive_records2().call()
        print('clinic2 records:')
        for record in all_records:
            print(record)
            print()
        print()
        all_records = contract_instance.functions.retrive_records3().call()
        print('clinic3 records:')
        for record in all_records:
            print(record)
            print()
            
                
        


In [436]:
def simulate_clinic1():
    c = clinic(0)
    c.add_new_patient('pat01', '20', '150 kg', '100 cm', 'M', '110 pb', '200 gh') #0
    c.add_new_patient('pat02', '21', '140 kg', '110 cm', 'f', '110 pb', '200 gh') #1
    c.regular_visit('0','55/120', '333', 'sick', 'panadol', 'non')
    c.lab_visit('1','55/80', '105', 'pcr', 'negative' )
    c.lab_visit('0','55/80', '105', 'pcr', 'positive' )

In [435]:
def simulate_clinic2():
    c = clinic(1)
    c.add_new_patient('pat1', '20', '150 kg', '100 cm', 'M', '110 pb', '200 gh') #0
    c.add_new_patient('pat2', '21', '140 kg', '110 cm', 'f', '110 pb', '200 gh') #1
    c.regular_visit('0','55/120', '333', 'sick', 'panadol', 'non')
    c.lab_visit('1','55/80', '105', 'pcr', 'negative' )
    c.lab_visit('0','55/80', '105', 'pcr', 'positive' )

In [434]:
def simulate_clinic3():
    c = clinic(2)
    c.add_new_patient('pat3', '50', '150 kg', '100 cm', 'M', '110 pb', '200 gh') #0
    c.add_new_patient('pat4', '60', '140 kg', '110 cm', 'f', '110 pb', '200 gh') #1
    c.regular_visit('0','55/120', '333', 'sick', 'sinovac', 'non')
    c.lab_visit('1','55/80', '105', 'blood_test', 'infected' )
    c.lab_visit('0','55/80', '105', 'blood_test', 'negative' )

### simulating multiple clinics simutaniously 

In [ ]:

t2 = threading.Thread(target=simulate_clinic1, args=())
t3 = threading.Thread(target=simulate_clinic3, args=())
t2.start()
t3.start()

In [439]:
c = clinic(2)
c.view_records('8')

records
['pat3 # 50 # 150 kg # 100 cm # M # 110 pb # 200 gh # 11/06/2022 18:12:09']


In [ ]:
c = clinic(0)
c.eval_view_all_records()